In [ ]:
import os
import errno
import csv
import random
import sys
import heapq
import re

from collections import Counter
from collections import OrderedDict
from scipy.optimize import curve_fit
from os import remove
from shutil import move


import plotly.plotly as py
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt

In [2]:
def selcheck(selection):
    
    with open('file/cdbrfss1999.csv', 'rt', encoding="ascii") as f:
    
        csvReader = csv.reader(f, delimiter = ',')
        header = csvReader.__next__()
        header = [item.lower()  for item in header]
        
        for choice in selection:
        
            if choice in header:
                print("{0} Found".format(choice))
            else:
                print("{0} NOT FOUND!".format(choice))
                print("{0} REMOVED!".format(choice))
                selection.remove(choice)
                          
                          
    return selection

In [3]:
def head_choice(header,selection):
    
    pushme = []
    
    for choice in selection:
        
        if choice in header:
            pushme.append(header.index(choice))
        else:
            print("HEADSHOT")
                          
                          
    return pushme

In [4]:
def csvappend(list,runs):
    with open('samples/sample' + str(runs) + '/sample.txt', 'rt', encoding="ascii") as csvinput:
        with open('samples/sample' + str(runs) + '/sample_bmi.txt', 'wt', encoding="ascii") as csvoutput:
        
            writer = csv.writer(csvoutput, lineterminator='\n')
            reader = csv.reader(csvinput)

            all = []
            ix = 0
            row = next(reader)
            row.append('BMI')
            all.append(row)

            for row in reader:
                row.append(list[ix])
                all.append(row)
                ix = ix + 1

            writer.writerows(all)

In [5]:
def typedet(strg):
    
    if isinstance(strg, (bool)):
        return "bool"
        print("bool")
    
    if strg.isspace() or not strg:
        
        return "null"
    
    
    try:
        int(strg)
        return "int"
    except ValueError:
        pass
    try:
        float(strg)
        return "float"
    except ValueError:
        pass
        
    return "str"

In [6]:
def count_it_ext(name_list,sol_file):
    
    ix = 0
    rowlength = 0
    
    typelist = [[] for k in range(len(name_list))]
    typehash = [[] for k in range(len(name_list))]
    
    print(len(typelist))

    with open('selected/selection.txt', 'rt', encoding="ascii") as f:
    
        csvReader = csv.reader(f, delimiter = ',')
        header = csvReader.__next__()
        header = [item.lower() for item in header]
        selection_indices = head_choice(header,name_list)
        print(selection_indices)
        
        rowlength = len(header)
        
        for row in csvReader:
            ix = ix + 1
            
            for i,idx in enumerate(selection_indices):
                typelist[i].append(typedet(row[idx]))
                typehash[i].append(row[idx])
        
    print("Events:\n")
    print("{0} cases with {1} cases\n".format(rowlength,ix))
    
    h = open(sol_file, 'w', encoding="ascii")
    
    h.write("Exercise 3.3 \n\n\n")
    h.write("There are {0} variables with {1} cases each.\n\n\n".format(rowlength,ix))
    
    h.write("Exercise 3.4-3.6\n\n\n")
    h.write("Remark to 3.4 - 3.6: We treat each numerical variable which has a range of values less than 10 as \"categorical\"\n")
    h.write("Since all the asked vars are of type \"int\" I neglect the other cases.\n\n\n")
    
    for i,idx in enumerate(typelist):
        
        exct = 4 + i
        
        d = Counter(typelist[i])
        md = max(d.keys());
        mdma = max(d.values())
        
        if md == "null" and len(d) > 1:
            md = heapq.nlargest(2,d.keys())[1]
            mdma = heapq.nlargest(2, d.values())[1]
            
        e = Counter(typehash[i])
        le = len(e)
        
        if "null" in d.keys():
            le = le - 1
        else:
            pass
        
        if le < 10:
            typus = "categorical,ordinal"
        else:
            typus = "numerical,discrete"
            
        h.write("Exercise 3." + str(exct) + "\n\n\n")
        h.write("Category \"{0}\" has entries: {1}.\n".format(name_list[i],d))
        h.write("Most frequent (non-null) event is \"{0}\" with {1} occurences.\n\n".format(md,mdma))
        h.write("Since there are {0} different (non-null) values, I assume the variable is of type {1}.\n\n".format(le,typus))
        h.write("-----------------------\n\n\n")
    
    d = Counter(typehash[0])
    
    dd = sorted(d.items())
    x_val = [int(x[0]) for x in dd]
    y_val = [int(x[1]) for x in dd]
    
    ax = plt.subplot(111)
    rects1 = ax.bar(x_val, y_val, align='center', width=.5)
    ax.set_title("GENHLTH")
    ax.set_ylabel('No of Occurences')
    ax.set_xlabel('Category')
    ax.set_xticks(np.arange(min(x_val), max(x_val)+1, 1.0))
    def autolabel(rects):
        for rect in rects:
            h = rect.get_height()
            ax.text(rect.get_x()+rect.get_width()/2., 1.05*h, '%d'%int(h),
                ha='center', va='bottom')
            
    autolabel(rects1)

    plt.savefig('samples/sample' + str(runs) + '/solution/' + str(name_list[0]) + '.png', bbox_inches='tight')
    
    plt.clf()
    
    smokey = []
    nonsmokey = []
    
    for ix,elem in enumerate(typehash[0]):
        
        if typedet(typehash[2] != "int"):
            next
        
        if typehash[2][ix] == "1":
            smokey.append(elem)
            
        if typehash[2][ix] == "2":
            nonsmokey.append(elem)
            
    
    d = Counter(smokey)
    dd = sorted(d.items())
    e = Counter(nonsmokey)
    ee = sorted(e.items())
    dx = np.asarray([float(x[0]) for x in dd])
    dy = [int(x[1]) for x in dd]
    ex = np.asarray([float(x[0]) for x in ee])
    ey = [int(x[1]) for x in ee]
    
    ax = plt.subplot(111)
    rects1 = ax.bar(dx-0.2, dy,width=0.3,color='b',align='center')
    rects2 = ax.bar(ex+0.2, ey,width=0.3,color='r',align='center')
    
    ax.set_ylabel('Scores')
    ax.legend( (rects1[0], rects2[0]), ('Smokerz', 'Non-Smokerz') )
    
    def autolabel(rects):
        for rect in rects:
            h = rect.get_height()
            ax.text(rect.get_x()+rect.get_width()/2., 1.05*h, '%d'%int(h),
                ha='center', va='bottom')
    
    autolabel(rects1)
    autolabel(rects2)
    
    plt.savefig('samples/sample' + str(runs) + '/solution/Smokey.png',bbox_inches='tight')
    
    
    #plt.show()
    plt.clf()
    
    h.close()
    f.close()

In [7]:
def coren_samples(selection,runs):
    
    try:
        os.makedirs("selected/")
    
    except OSError as exception:
        if exception.errno != errno.EEXIST:
            raise

    g = open('selected/selection.txt', 'w', encoding="ascii")

    with open('samples/sample' + str(runs) + '/sample.txt', 'rt', encoding="ascii") as f:
    
        csvReader = csv.reader(f, delimiter = ',')
        
        header = csvReader.__next__()
        header = [item.lower() for item in header]
        
        for head in header:
            re.sub('[^A-Za-z0-9]+', '', head)
        
        selection_indices = head_choice(header,selection)
        print(header)
        print(selection_indices)
        
        for ix,hed in enumerate(selection):
            
            if ix > 0:
                g.write(",")
                
            g.write("{0}".format(hed))
            
        g.write("\n")
        
        ix = 0
        
        #BMI 2,3 - 6
        
        bmilist = []
        
        
        idx = 0
        
        for row in csvReader:
            
            idx = idx + 1
            
            
            
            for ix,val in enumerate(selection_indices):
                
                if ix > 0:
                    g.write(",")
                
                g.write("{0}".format(row[val]))
        
            g.write("\n")
            
            kg1 = selection_indices[2]
            kg2 = selection_indices[3]
            g1 = selection_indices[5]
            bmi_max = 100
            
            
            if((typedet(row[kg1]) == "int") and (typedet(row[kg2]) == "int") and (typedet(row[g1]) == "int")):

                in_ht = 12*int(row[kg1]) + int(row[kg2])
                #print(in_ht)
                #print(int(row[g1]))
                #print("==========")
                bmi = round(703*int(row[g1])/(in_ht**2),2)
                if bmi > bmi_max:
                    bmi = " "
                bmilist.append(bmi)
               
            else:
                bmilist.append(" ")
        
        bmilist_clc = np.asarray([x for x in bmilist if x != " "])
        
        mybins = 200
        
        print("MAXIMUM")
        print(max(bmilist_clc))
        
        s = pd.Series(bmilist_clc)
        p = s.plot(kind='hist', bins=mybins, color='orange')
        
        
        p.set_title("BMI < 150, " + str(mybins) + " bins")
        p.set_ylabel('Absolute Frequency')
        p.set_xlabel('BMI')
        
        plt.savefig("samples/sample" + str(runs) + "/solution/BMIbarchart.png", bbox_inches="tight")
        plt.clf()
        
        
        ser = bmilist_clc

        plt.hist(ser, normed=True, bins=np.arange(0, bmi_max , 1), label = "BMI Bar Chart")
        plt.title = ("Gaussian of BMI Bar Chart, Bins:")

        xt = plt.xticks()[0]  
        xmin, xmax = min(xt), max(xt)  
        lnspc = np.linspace(xmin, xmax, bmi_max)

        m, s = stats.norm.fit(ser)
        pdf_g = stats.norm.pdf(lnspc, m, s)  
        plt.plot(lnspc, pdf_g, label="Gauss with\n mu = " + str(round(m,2)) + "\n sigma = " + str(round(s,2)))
        
        plt.xlabel=("BMI")
        plt.ylabel = ("relative frequency")      
        plt.title = ("Gaussian of BMI Bar Chart, Bins:")
        
        plt.legend()

        
        plt.savefig("samples/sample"+str(runs)+"/solution/BMIgauss.png", bbox_inches="tight")
        
        plt.clf()
        
        
        
    f.close()
    g.close()
    
    
    csvappend(bmilist,runs)

In [ ]:
def collect_samples(runs):
    
    samplesize = 20000
    maxindex = 159988
    

    g = open('samples/sample' + str(runs) + '/sample.txt', 'w', encoding="ascii")

    box = sorted(random.sample(range(maxindex), k=samplesize))


    with open('file/cdbrfss1999.csv', 'rt', encoding="ascii") as f:
    
        csvReader = csv.reader(f, delimiter = ',')
        header = csvReader.__next__()
        
        for ix,head in enumerate(header):
     
            
            if ix == len(header)-1:
                g.write("{0}\n".format(header[ix]))
            else:
                g.write("{0},".format(header[ix]))

        for row in csvReader:
        
            if csvReader.line_num - 2 in box:
                
                for ix,elem in enumerate(row):
                    
                    if ix == len(row)-1:
                        g.write("{0}\n".format(row[ix]))
                    else:
                        g.write("{0},".format(row[ix]))
        
        
    
    f.close()
    g.close()
    
    return header

In [ ]:
conti = "y"
runs = 1
selection = ["genhlth","exerany","htf","hti","smoke100","height","weight","wtdesire","age","sex"]
name_list = ["genhlth","weight","smoke100"]
header = []

selection = selcheck(selection)

try:
    os.makedirs("solution")
    
except OSError as exception:
    if exception.errno != errno.EEXIST:
        raise
        
try:
    os.makedirs("samples")
    
except OSError as exception:
    if exception.errno != errno.EEXIST:
        raise
            
sol_file = 'solution/solution.txt'

while conti == "y":
    
    try:
        os.makedirs("samples/sample"+ str(runs))
    
    except OSError as exception:
        if exception.errno != errno.EEXIST:
            raise
            
    try:
        os.makedirs("samples/sample"+ str(runs) + "/solution")
    
    except OSError as exception:
        if exception.errno != errno.EEXIST:
            raise
   

    header = collect_samples(runs)
    
    coren_samples(selection,runs)
    
    count_it_ext(name_list,sol_file)
    
    conti = input("Another sample? (\"y\" for yes)")
    
    runs = runs + 1


print("Finito")

genhlth Found
exerany Found
htf Found
hti Found
smoke100 Found
height NOT FOUND!
height REMOVED!
wtdesire Found
age Found
sex Found
['state', 'geostr', 'denstr', 'psu', 'record', 'imonth ', 'iday ', 'iyear ', 'intvid ', '', 'dispcode', 'winddown', 'seqno', 'nattmpts', 'nrecsel', 'nrecstr', 'bpselwt', 'wakss1s2', 'numadult', 'nummen', 'numwomen', 'genhlth', 'physhlth', 'menthlth', 'poorhlth', 'hlthplan', 'medicar2', 'typcovr1', 'typcovr2', 'nocov12', 'pastplan', 'medcost', 'checkup', 'bptake', 'bphigh', 'highgt1', 'bloodcho', 'cholchk', 'toldhi', 'diabetes', 'lastden2', 'rmvteeth', 'denclean', 'sunburn', 'numburn', 'smoke100', 'smokeday', 'smokenum', 'smoknm30', 'stopsmok', 'lastsmok', 'drinkany', 'alcohol', 'nalcocc', 'drinkge5', 'drinkdri', 'age', 'orace', 'hispanic', 'marital', 'chld04', 'chld0512', 'chld1317', 'educa', 'employ', 'income2', 'weight', 'htf', 'hti', 'ctycode', 'numhhold', 'numphons', 'sex', 'hadmam', 'howlong', 'whydone', 'profexam', 'lengexam', 'reasexam', 'hadpap', '